# Ally Financial (ALLY) Stock Valuation Analysis

This notebook provides a comprehensive valuation analysis of Ally Financial using multiple methods:

1. **Book Value** - Basic equity valuation
2. **Adjusted Book Value** - Equity minus intangibles
3. **P/E Ratio** - Earnings-based valuation
4. **Dividend Discount Model (DDM)** - Present value of future dividends
5. **Comparable Companies** - Peer multiples analysis
6. **DCF (Free Cash Flow)** - Discounted cash flow valuation
7. **LSTM Neural Network** - AI-based price prediction
8. **Monte Carlo Simulation** - Probabilistic price forecasting

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plotting style
try:

    plt.style.use('seaborn-v0_8-whitegrid')
except OSError:
    plt.style.use('seaborn-whitegrid')
sns.set_palette('husl')

print("Libraries loaded successfully!")

In [ ]:
# Import our custom modules
from src.data_loader import DataLoader
from src.valuation import ValuationEngine
from src.monte_carlo import MonteCarloSimulation

print("Custom modules loaded successfully!")

## 1. Data Loading and Exploration

In [ ]:
# Initialize data loader for Ally Financial
loader = DataLoader(ticker="ALLY")

# Get company summary
summary = loader.get_summary()
print("=" * 50)
print("ALLY FINANCIAL - KEY METRICS")
print("=" * 50)
for key, value in summary.items():
    if isinstance(value, float):
        if abs(value) > 1e6:
            print(f"{key}: ${value:,.0f}")
        else:
            print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")

In [ ]:
# Get historical price data
prices = loader.get_historical_prices(period="5y")
print(f"Historical data shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")
prices.tail()

In [ ]:
# Plot historical prices
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Price chart
axes[0].plot(prices.index, prices['Close'], label='Close Price', linewidth=1.5)
axes[0].fill_between(prices.index, prices['Low'], prices['High'], alpha=0.3, label='Daily Range')
axes[0].set_title('ALLY - Historical Stock Price', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volume chart
axes[1].bar(prices.index, prices['Volume'], alpha=0.7, label='Volume')
axes[1].set_title('ALLY - Trading Volume', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Volume')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Traditional Valuation Methods

In [ ]:
# Initialize valuation engine
valuation = ValuationEngine(data_loader=loader)

### 2.1 Book Value Valuation

In [ ]:
book_value = valuation.book_value_valuation()
print("=" * 50)
print("BOOK VALUE ANALYSIS")
print("=" * 50)
for key, value in book_value.items():
    if isinstance(value, float):
        if abs(value) > 1e6:
            print(f"{key}: ${value:,.0f}")
        else:
            print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

### 2.2 Adjusted Book Value Valuation

In [ ]:
adj_book_value = valuation.adjusted_book_value_valuation()
print("=" * 50)
print("ADJUSTED BOOK VALUE ANALYSIS")
print("=" * 50)
for key, value in adj_book_value.items():
    if isinstance(value, float):
        if abs(value) > 1e6:
            print(f"{key}: ${value:,.0f}")
        else:
            print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

### 2.3 P/E Ratio Valuation

In [ ]:
pe_valuation = valuation.pe_ratio_valuation()
print("=" * 50)
print("P/E RATIO ANALYSIS")
print("=" * 50)
for key, value in pe_valuation.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

### 2.4 Dividend Discount Model (DDM)

In [ ]:
ddm = valuation.dividend_discount_model(
    required_return=0.10,  # 10% required return
    growth_rate=0.03,      # 3% dividend growth
    terminal_growth=0.02,  # 2% perpetual growth
    forecast_years=5
)
print("=" * 50)
print("DIVIDEND DISCOUNT MODEL ANALYSIS")
print("=" * 50)
for key, value in ddm.items():
    if key == 'projected_dividends':
        print(f"\n{key}:")
        for div in value:
            print(f"  Year {div['year']}: Dividend=${div['dividend']:.2f}, PV=${div['pv']:.2f}")
    elif isinstance(value, float):
        if abs(value) > 1e6:
            print(f"{key}: ${value:,.0f}")
        else:
            print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

### 2.5 Comparable Companies Analysis

In [ ]:
comparables = valuation.comparable_companies_valuation()
print("=" * 50)
print("COMPARABLE COMPANIES ANALYSIS")
print("=" * 50)

# Display peer metrics table
if 'peer_metrics' in comparables and comparables['peer_metrics']:
    peer_df = pd.DataFrame(comparables['peer_metrics'])
    print("\nPeer Company Metrics:")
    display(peer_df)

print(f"\nMedian P/E: {comparables.get('median_pe', 'N/A')}")
print(f"Median P/B: {comparables.get('median_pb', 'N/A')}")
print(f"Implied Value (P/E): ${comparables.get('implied_value_pe', 0):.2f}")
print(f"Implied Value (P/B): ${comparables.get('implied_value_pb', 0):.2f}")
print(f"Average Intrinsic Value: ${comparables.get('intrinsic_value', 0):.2f}")
print(f"Current Price: ${comparables.get('current_price', 0):.2f}")
print(f"Upside Potential: {comparables.get('upside_potential', 0):.2f}%")

### 2.6 DCF (Free Cash Flow) Valuation

In [ ]:
dcf = valuation.dcf_fcf_valuation(
    required_return=0.10,  # 10% WACC
    growth_rate=0.05,      # 5% FCF growth
    terminal_growth=0.02,  # 2% perpetual growth
    forecast_years=5
)
print("=" * 50)
print("DCF (FREE CASH FLOW) ANALYSIS")
print("=" * 50)
for key, value in dcf.items():
    if key == 'projected_fcfs':
        print(f"\n{key}:")
        for fcf in value:
            print(f"  Year {fcf['year']}: FCF=${fcf['fcf']:,.0f}, PV=${fcf['pv']:,.0f}")
    elif isinstance(value, float):
        if abs(value) > 1e6:
            print(f"{key}: ${value:,.0f}")
        else:
            print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

## 3. Valuation Comparison

In [ ]:
# Get summary of all valuations
valuation_summary = valuation.get_valuation_summary()
print("\n" + "=" * 70)
print("VALUATION METHODS COMPARISON")
print("=" * 70)
display(valuation_summary)

In [ ]:
# Get weighted fair value estimate
fair_value = valuation.get_fair_value_estimate()
print("\n" + "=" * 50)
print("WEIGHTED FAIR VALUE ESTIMATE")
print("=" * 50)
print(f"Fair Value Estimate: ${fair_value['fair_value_estimate']:.2f}")
print(f"Current Price: ${fair_value['current_price']:.2f}")
print(f"Upside Potential: {fair_value['upside_potential']:.2f}%")
print(f"Methods Used: {', '.join(fair_value['valid_methods_used'])}")

In [ ]:
# Visualize valuation comparison
fig, ax = plt.subplots(figsize=(12, 6))

# Filter out methods with zero or error values
valid_valuations = valuation_summary[valuation_summary['Intrinsic Value ($)'] > 0].copy()

methods = valid_valuations['Method']
values = valid_valuations['Intrinsic Value ($)']
current_price = valid_valuations['Current Price ($)'].iloc[0]

# Create bar chart
colors = ['#2ecc71' if v > current_price else '#e74c3c' for v in values]
bars = ax.bar(methods, values, color=colors, alpha=0.8, edgecolor='black')

# Add current price line
ax.axhline(y=current_price, color='blue', linestyle='--', linewidth=2, label=f'Current Price: ${current_price:.2f}')

# Add fair value line
ax.axhline(y=fair_value['fair_value_estimate'], color='purple', linestyle=':', linewidth=2, 
           label=f"Fair Value: ${fair_value['fair_value_estimate']:.2f}")

# Customize chart
ax.set_ylabel('Price ($)', fontsize=12)
ax.set_title('ALLY - Valuation Methods Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')

# Add value labels on bars
for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            f'${val:.2f}', ha='center', va='bottom', fontsize=10)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 4. Monte Carlo Simulation

In [ ]:
# Initialize Monte Carlo simulator
mc = MonteCarloSimulation(
    n_simulations=10000,
    n_days=252,  # 1 year
    random_seed=42
)

# Run simulation
close_prices = prices['Close']
mc_results = mc.run_simulation(close_prices)

# Display summary statistics
mc_summary = mc.get_summary_statistics(mc_results)
print("\n" + "=" * 50)
print("MONTE CARLO SIMULATION RESULTS")
print("=" * 50)
display(mc_summary)

In [ ]:
# Plot Monte Carlo simulation results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Sample price paths
ax1 = axes[0, 0]
n_paths_to_plot = 100
for i in range(n_paths_to_plot):
    ax1.plot(mc_results['price_paths'][i], alpha=0.1, color='blue')
ax1.plot(np.mean(mc_results['price_paths'], axis=0), color='red', linewidth=2, label='Mean Path')
ax1.axhline(y=mc_results['initial_price'], color='green', linestyle='--', label=f"Initial: ${mc_results['initial_price']:.2f}")
ax1.set_title('Monte Carlo Price Paths (1 Year)', fontsize=12, fontweight='bold')
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Price ($)')
ax1.legend()

# Plot 2: Distribution of final prices
ax2 = axes[0, 1]
ax2.hist(mc_results['final_prices'], bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
ax2.axvline(x=mc_results['initial_price'], color='green', linestyle='--', linewidth=2, label=f"Initial: ${mc_results['initial_price']:.2f}")
ax2.axvline(x=mc_results['mean_final_price'], color='red', linestyle='-', linewidth=2, label=f"Mean: ${mc_results['mean_final_price']:.2f}")
ax2.axvline(x=mc_results['percentiles'][5], color='orange', linestyle=':', linewidth=2, label=f"5th%: ${mc_results['percentiles'][5]:.2f}")
ax2.axvline(x=mc_results['percentiles'][95], color='orange', linestyle=':', linewidth=2, label=f"95th%: ${mc_results['percentiles'][95]:.2f}")
ax2.set_title('Distribution of Final Prices', fontsize=12, fontweight='bold')
ax2.set_xlabel('Price ($)')
ax2.set_ylabel('Density')
ax2.legend(fontsize=8)

# Plot 3: Confidence intervals
ax3 = axes[1, 0]
intervals = mc.get_confidence_intervals(mc_results['price_paths'])
days = range(mc_results['price_paths'].shape[1])

ax3.fill_between(days, intervals['95%']['lower'], intervals['95%']['upper'], alpha=0.2, color='blue', label='95% CI')
ax3.fill_between(days, intervals['90%']['lower'], intervals['90%']['upper'], alpha=0.3, color='blue', label='90% CI')
ax3.plot(days, intervals['95%']['mean'], color='red', linewidth=2, label='Mean')
ax3.axhline(y=mc_results['initial_price'], color='green', linestyle='--', label='Initial Price')
ax3.set_title('Price Forecast with Confidence Intervals', fontsize=12, fontweight='bold')
ax3.set_xlabel('Trading Days')
ax3.set_ylabel('Price ($)')
ax3.legend()

# Plot 4: Return distribution
ax4 = axes[1, 1]
returns = (mc_results['final_prices'] - mc_results['initial_price']) / mc_results['initial_price'] * 100
ax4.hist(returns, bins=50, density=True, alpha=0.7, color='purple', edgecolor='black')
ax4.axvline(x=0, color='black', linestyle='-', linewidth=2)
ax4.axvline(x=np.mean(returns), color='red', linestyle='--', linewidth=2, label=f"Mean: {np.mean(returns):.1f}%")
ax4.set_title('Distribution of 1-Year Returns', fontsize=12, fontweight='bold')
ax4.set_xlabel('Return (%)')
ax4.set_ylabel('Density')
ax4.legend()

plt.tight_layout()
plt.show()

## 5. LSTM Price Prediction

In [ ]:
# Import LSTM model (requires TensorFlow)
try:
    from src.lstm_model import LSTMPredictor
    
    # Initialize and train LSTM model
    lstm = LSTMPredictor(
        sequence_length=60,
        units=50,
        epochs=25,  # Reduced for faster execution
        batch_size=32
    )
    
    print("Training LSTM model...")
    training_results = lstm.train(close_prices, train_ratio=0.8, verbose=1)
    
    print(f"\nTraining RMSE: ${training_results['train_rmse']:.2f}")
    print(f"Test RMSE: ${training_results['test_rmse']:.2f}")
    
    LSTM_AVAILABLE = True
except ImportError as e:
    print(f"LSTM model not available: {e}")
    print("Install TensorFlow to enable LSTM predictions: pip install tensorflow")
    LSTM_AVAILABLE = False

In [ ]:
if LSTM_AVAILABLE:
    # Predict future prices
    days_ahead = 30
    future_predictions = lstm.predict_future(close_prices, days_ahead=days_ahead)
    future_dates = lstm.get_prediction_dates(close_prices.index[-1], days_ahead=days_ahead)
    
    # Plot LSTM predictions
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Plot 1: Training results
    ax1 = axes[0]
    ax1.plot(training_results['actual_test'], label='Actual', color='blue', linewidth=1.5)
    ax1.plot(training_results['test_predictions'], label='LSTM Prediction', color='red', linestyle='--', linewidth=1.5)
    ax1.set_title('LSTM Model - Test Set Performance', fontsize=12, fontweight='bold')
    ax1.set_xlabel('Time Steps')
    ax1.set_ylabel('Price ($)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Future predictions
    ax2 = axes[1]
    # Plot last 90 days of historical data
    historical_end = close_prices.iloc[-90:]
    ax2.plot(historical_end.index, historical_end.values, label='Historical', color='blue', linewidth=1.5)
    ax2.plot(future_dates, future_predictions, label='LSTM Forecast', color='red', linestyle='--', linewidth=2)
    ax2.axvline(x=close_prices.index[-1], color='gray', linestyle=':', label='Forecast Start')
    ax2.set_title(f'LSTM - {days_ahead}-Day Price Forecast', fontsize=12, fontweight='bold')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Price ($)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print prediction summary
    print("\n" + "=" * 50)
    print("LSTM PREDICTION SUMMARY")
    print("=" * 50)
    print(f"Current Price: ${close_prices.iloc[-1]:.2f}")
    print(f"Predicted Price ({days_ahead} days): ${future_predictions[-1]:.2f}")
    print(f"Predicted Change: {((future_predictions[-1] - close_prices.iloc[-1]) / close_prices.iloc[-1] * 100):.2f}%")

## 6. Final Summary and Recommendations

In [ ]:
# Create comprehensive summary
print("\n" + "=" * 70)
print("ALLY FINANCIAL - COMPREHENSIVE VALUATION SUMMARY")
print("=" * 70)

current_price = loader.get_current_price()
print(f"\nCurrent Market Price: ${current_price:.2f}")

print("\n--- Traditional Valuation Methods ---")
print(valuation_summary.to_string(index=False))

print(f"\n--- Weighted Fair Value Estimate ---")
print(f"Fair Value: ${fair_value['fair_value_estimate']:.2f}")
print(f"Upside/Downside: {fair_value['upside_potential']:.2f}%")

print(f"\n--- Monte Carlo Simulation (1-Year) ---")
print(f"Mean Predicted Price: ${mc_results['mean_final_price']:.2f}")
print(f"95% Confidence Interval: ${mc_results['percentiles'][5]:.2f} - ${mc_results['percentiles'][95]:.2f}")
print(f"Probability of Profit: {mc_results['prob_profit']:.1f}%")

if LSTM_AVAILABLE:
    print(f"\n--- LSTM Prediction ({days_ahead}-Day) ---")
    print(f"Predicted Price: ${future_predictions[-1]:.2f}")
    print(f"Predicted Change: {((future_predictions[-1] - current_price) / current_price * 100):.2f}%")

# Investment recommendation
print("\n" + "=" * 70)
print("INVESTMENT ANALYSIS")
print("=" * 70)

if fair_value['upside_potential'] > 20:
    recommendation = "STRONG BUY - Significantly undervalued"
elif fair_value['upside_potential'] > 10:
    recommendation = "BUY - Moderately undervalued"
elif fair_value['upside_potential'] > -10:
    recommendation = "HOLD - Fairly valued"
elif fair_value['upside_potential'] > -20:
    recommendation = "SELL - Moderately overvalued"
else:
    recommendation = "STRONG SELL - Significantly overvalued"

print(f"\nRecommendation: {recommendation}")
print(f"\nNote: This analysis is for educational purposes only and should not be")
print(f"considered as financial advice. Always conduct your own research and")
print(f"consult with a financial advisor before making investment decisions.")

In [ ]:
# Final visualization: All methods comparison
fig, ax = plt.subplots(figsize=(14, 8))

# Prepare data
methods_data = {
    'Book Value': book_value.get('intrinsic_value', 0),
    'Adjusted Book': adj_book_value.get('intrinsic_value', 0),
    'P/E Ratio': pe_valuation.get('intrinsic_value', 0),
    'DDM': ddm.get('intrinsic_value', 0),
    'Comparables': comparables.get('intrinsic_value', 0),
    'DCF/FCF': dcf.get('intrinsic_value', 0),
    'Monte Carlo Mean': mc_results['mean_final_price'],
    'Fair Value Est.': fair_value['fair_value_estimate']
}

# Filter out zero values
methods_data = {k: v for k, v in methods_data.items() if v > 0}

methods = list(methods_data.keys())
values = list(methods_data.values())

# Create bar chart
colors = ['#3498db' if v > current_price else '#e74c3c' for v in values]
bars = ax.barh(methods, values, color=colors, alpha=0.8, edgecolor='black')

# Add current price line
ax.axvline(x=current_price, color='green', linestyle='--', linewidth=3, 
           label=f'Current Price: ${current_price:.2f}')

# Add value labels
for bar, val in zip(bars, values):
    ax.text(val + 0.5, bar.get_y() + bar.get_height()/2, 
            f'${val:.2f}', va='center', fontsize=10)

ax.set_xlabel('Price ($)', fontsize=12)
ax.set_title('ALLY - Complete Valuation Analysis', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=11)
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()